<a href="https://colab.research.google.com/github/Rushaida0077/Breast-Cancer-Diagnosis-Prediction/blob/main/CSE_301_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

**Load Dataset**

In [13]:
B_cancer_data = pd.read_csv("/content/drive/MyDrive/Dataset.csv")
print("Initial Data:")
B_cancer_data.head()

Initial Data:


,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


**Drop unwanted columns**

In [14]:
B_cancer_data = B_cancer_data.drop(columns=["id", "Unnamed: 32"])

**Encode diagnosis (M → 1, B → 0)**

In [15]:
B_cancer_data["diagnosis"] = B_cancer_data["diagnosis"].map({"M": 1, "B": 0})
print("\nCounts of each class:")
print(B_cancer_data["diagnosis"].value_counts())



Counts of each class:
diagnosis
0    357
1    212
Name: count, dtype: int64


In [16]:
B_cancer_data.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


**Min-Max Normalization**

In [17]:
features = B_cancer_data.drop("diagnosis", axis=1)
target = B_cancer_data["diagnosis"]

scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features)

**Split Train (80%) / Test (20%)**

In [18]:
X_train, X_test, y_train, y_test = train_test_split(
    features_scaled, target, test_size=0.2, random_state=42
)

**Apply K-Means Clustering (k = 2)**

In [20]:
kmeans_model = KMeans(n_clusters=2, random_state=42)
cluster_labels = kmeans_model.fit_predict(features_scaled)
B_cancer_data["cluster"] = cluster_labels

In [21]:
cluster_comparison = pd.crosstab(B_cancer_data["cluster"], B_cancer_data["diagnosis"])
print("\nCluster vs Diagnosis:")
print(cluster_comparison)



Cluster vs Diagnosis:
diagnosis    0    1
cluster            
0            9  180
1          348   32


**Train KNN Classifier (k=5)**

In [22]:
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, y_train)

predictions = knn_model.predict(X_test)


**Evaluate Performance**

In [24]:
accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)
f1 = f1_score(y_test, predictions)

print("\nModel Performance:")
print(f"Accuracy : {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")
print(f"F1-score : {f1:.4f}")



Model Performance:
Accuracy : 0.9649
Precision: 0.9535
Recall   : 0.9535
F1-score : 0.9535
